# Deep Learning
# DL06 PM2.5 Prediccion MLP-SOLUCION

## <font color='blue'>**Multilayer preceptron aplicado a series de tiempo univariadas**</font>


Una serie de tiempo es una serie de puntos de datos indexados (o listados o graficados) en orden de tiempo. Más comúnmente, una serie de tiempo es una secuencia tomada en sucesivos puntos equidistantes en el tiempo. Por lo tanto, es una secuencia de datos de tiempo discreto. Ejemplos de series temporales son las alturas de las mareas oceánicas, los recuentos de manchas solares y el valor de cierre diario del Promedio Industrial Dow Jones.

Las series temporales se trazan con mucha frecuencia a través de gráficos de líneas. Las series de tiempo se usan en estadística, procesamiento de señales, reconocimiento de patrones, econometría, finanzas matemáticas, pronóstico del tiempo, predicción de terremotos, electroencefalografía, ingeniería de control, astronomía, ingeniería de comunicaciones y en gran medida en cualquier dominio de la ciencia aplicada y la ingeniería que involucra mediciones temporales.

El término __serie temporal univariadas__, se refiere a una serie temporal que consiste en observaciones individuales (escalares) registradas secuencialmente en incrementos de tiempo iguales.
    
En este notebook, utilizaremos un perceptrón multicapa para desarrollar modelos de pronóstico de series temporales univariadas.
El conjunto de datos utilizado para los ejemplos de este notebook es sobre la contaminación del aire medida por la concentración de material particulado (PM) de diámetro menor o igual a 2.5 micrómetros. Hay otras variables
tales como presión de aire, temperatura del aire, punto de rocío, que tambien serán utilizadas para realizar predicciones.


En este caso, se desarrollará un modelo de series temporales: para la predicción de pm2.5.

El notebook se divide en las siguientes etapas:
1. Visualización de la data
2. Procesamiento de la data
3. Construcción del modelo y su entrenamiento con Keras
4. Resultados y validación del modelo.


Importancia de la predicción del material particulado:

El material particulado respirable presente en la atmósfera de nuestras ciudades en forma sólida o líquida (polvo, cenizas, hollín, partículas metálicas, cemento y polen, entre otras) se puede dividir, según su tamaño, en dos grupos principales. A las de diámetro aerodinámico igual o inferior a los 10 µm o 10 micrómetros (1 $\mu m$ corresponde a la milésima parte de un milímetro) se las denomina PM10 y a la fracción respirable más pequeña, PM2,5. Estas últimas están constituidas por aquellas partículas de diámetro aerodinámico inferior o igual a los 2,5 micrómetros, es decir, son 100 veces más delgadas que un cabello humano.


El conjunto de datos se ha descargado del repositorio de aprendizaje automático de UCI.

https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data

In [ ]:
#Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

### Etapa 1. Visualización de la data

En esta etapa nos hacemos una idea de como es la distribución de la data. Entre que valores fluctua,  en el caso de las series de tiempo univariadas, es natural realizar Boxplots y graficos de tiempo. Las librerías tipicas que utilizaremos en esta etapa son: `pandas`, `matplotlib`, `numpy` y `seaborn`.


In [ ]:
from __future__ import print_function
import os
import sys
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import datetime

In [ ]:
#set current working directory
path = '/content/drive/MyDrive/Curso/Industria Inteligente/2023-2S/Datos/'
os.chdir(path)

In [ ]:
#Read the dataset into a pandas.DataFrame
df = pd.read_csv('PRSA_data_2010.1.1-2014.12.31.csv')

In [ ]:
#Cargar archivo subido
df = pd.read_csv('PRSA_data_2010.1.1-2014.12.31.csv')

## <font color='green'>**Actividad 1**</font>

En esta actividad queremos visualizar nuestra data.  (30 minutos)

1. ¿Cuantos valores nulos tenemos?, como los podemos tratar. ¿Probemos borrando data?

2. Cree una columna que se llame datetime y ordenelo en forma ascendente. Puede mirar el siguiente código para inspirarse.

```python
df['datetime'] = df[['year', 'month', 'day', 'hour']].apply(lambda row: datetime.datetime(year=row['year'], month=row['month'], day=row['day'],
                                                                                          hour=row['hour']), axis=1)
df.sort_values('datetime', ascending=True, inplace=True)
```

3. Visualice la serie de tiempo para la variable pm2.5. Utilice gráficos de línea, box plots e histogramas. Visualice distintos periodos de tiempo, se observa el tamaño del ciclo?



In [ ]:
print('Shape of the dataframe:', df.shape)
print(df['pm2.5'].isna().sum())

In [ ]:
#Veamos las primeras 5 líneas del dataframe
df.head()

### Datos faltantes:

En los datos de series de tiempo, si faltan valores, hay dos formas de tratar los datos incompletos:

1. Omita todo el registro que contiene información.
2. Imputar la información que falta.

Dado que los datos de series temporales tienen propiedades temporales, solo algunas de las metodologías estadísticas son apropiadas para los datos de series temporales.

Metodos elementales de imputación:
1. Con la media
2. Con la mediana
3. Con la moda
4. Calcular el valor apropiado y reeemplazar los NAs
5. Usar modelos estadísticos y de Machine Leaning (e.g. K-NN).

<font color='green'>**Fin Actividad 1**</font>

### Etapa 2. Preprocesamiento de la data

En esta segunda etapa preparamos los datos con el objetivo de realizar un entrenamiento robusto de nuestra red neuronal. Usualmente los datos como primera etapa se normalizan. Empiricamente se ha observando que los datos normalizados (No siempre) generan modelos de clasificiación y de regresión con mejores metricas que los no normalizadas. Por otra parte, Los algoritmos de descenso de gradiente funcionan mejor (por ejemplo, convergen más rápido) si las variables están dentro del rango $[-1, 1]$. Muchas fuentes relajan el límite incluso $[-3, 3]$.

Posteriormente los datos debens ser separados en tres conjuntos: Entrenamiento, validación y test. Usualmente el ultimo de test se utiliza con una prueba nueva de datos. Finalmente debemos construir el conjunto de vectores que serán utilizados para entrenar la red neuronal perceptron multicapa. Las librerias. Adicionalmente en esta sección utilizaremos la librería `sklearn` para realizar la normalización.


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df['scaled_pm2.5'] = scaler.fit_transform(np.array(df['pm2.5']).reshape(-1, 1)) # Nos lo deja como vector.

In [ ]:
import joblib

# Guardemos el escalador
fileoutScaler = 'scaler_model.sav'
joblib.dump(scaler, fileoutScaler)


<p style='text-align: justify;'>
Antes de entrenar el modelo, el conjunto de datos se divide en dos partes: conjunto de entrenamiento y conjunto de validación.
La red neuronal se entrena en el conjunto de entrenamiento. Esto significa el cálculo de la función de pérdida, backpropagation
y los pesos actualizados por un algoritmo de descenso de gradiente se realizan en el conjunto de entrenamiento. El conjunto de validación se utiliza para evaluar el modelo y para determinar el número de epochs en el entrenamiento del modelo. Aumentando el número de epochs disminuirán aún más la función de pérdida en el conjunto de entrenamiento, pero es posible que no necesariamente tengan el mismo efecto para el conjunto de validación debido al sobreajuste en el conjunto de entrenamiento. Utilizamos Keras con el backend Tensorflow para definir y entrenar el modelo. Todos los pasos involucrados en la capacitación y validación del modelo se realizan llamando a las funciones apropiadas de la API de Keras.
 </p>

In [ ]:
"""
Comencemos dividiendo el conjunto de datos en entrenamiento y validación. El período de tiempo del conjunto de datos si es de
1 de enero de 2010 al 31 de diciembre de 2014. Los primeros cuatro años: 2010 a 2013 se utiliza como entrenamiento y
2014 se mantiene para validación.
"""
# Utilizamos pandas para realizar este proceso.
split_date = datetime.datetime(year=2014, month=1, day=1, hour=0)
df_train = df.loc[df['datetime'] < split_date]
df_val = df.loc[df['datetime'] >= split_date]
print('Shape of train:', df_train.shape)
print('Shape of test:', df_val.shape)

In [ ]:
#Miremos nuestro conjunto de entrenamiento
df_train.head()

In [ ]:
#Miremos nuestro conjunto de validación
df_val.head()

In [ ]:
#Vamos a resetar los indices para ser ordenados
df_val.reset_index(drop=True, inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Asumiendo que 'df', 'df_train' y 'df_val' son tus DataFrames y tienen las columnas necesarias

"""
El conjunto de entrenamiento y validación lo volvemos a dibujar.
"""

plt.figure(figsize=(16, 5.5))
g = sns.lineplot(x=df['datetime'], y=df_train['scaled_pm2.5'], color='b')
g.set_title('Serie de tiempo del conjunto de entrenamiento pm2.5 escalada')
g.set_xlabel('Index')
g.set_ylabel('Valores escalados de pm2.5')

plt.figure(figsize=(16, 5.5))
g = sns.lineplot(x=df['datetime'], y=df_val['scaled_pm2.5'], color='r')
g.set_title('Serie de tiempo del conjunto de validación pm2.5 escalada')
g.set_xlabel('Index')
g.set_ylabel('Valores escalados de pm2.5')



Ahora necesitamos generar vectores ($X$) y una variable objetivo ($y$) para entrenar y validar. La matriz de los regresores o variables independientes y la matriz  de la variable dependiente se crean a partir de la matriz 1-D original de la columna `scaled_pm2.5`. Para el modelo de pronóstico de series de tiempo, los últimos siete días de observaciones se utilizan para predecir el día siguiente, este valor se estudia y se pueden ejercitar distintas ventanas de tiempo. el dia 7 surge de la observación de los gráficos de linea.  Definimos una función que toma la serie de tiempo original y el número de pasos de tiempo en los regresores como entrada para generar las matrices de $X$ e $y$.

In [ ]:
def makeXy(ts, nb_timesteps):
    """
    Input:
           ts: serie de tiempo original.
           nb_timesteps: Ventana de tiempo.
    Output:
           X: 2-D array de los regresores o variables independientes.
           y: 1-D array de variable dependiente.
    """
    X = []
    y = []
    for i in range(nb_timesteps, ts.shape[0]):
        X.append(list(ts.loc[i-nb_timesteps:i-1]))
        y.append(ts.loc[i])
    X, y = np.array(X), np.array(y) # Lo tranformamos a nparray
    return X, y

In [ ]:
X_train, y_train = makeXy(df_train['scaled_pm2.5'], 7)
print('Shape of train arrays:', X_train.shape, y_train.shape)

In [ ]:
X_val, y_val = makeXy(df_val['scaled_pm2.5'], 7)
print('Shape of validation arrays:', X_val.shape, y_val.shape)

### Etapa 3. Definiendo el modelo de perceptron de multicapa en Keras
<p style='text-align: justify;'>
Un solo perceptrón solo se puede utilizar para implementar funciones separables linealmente. Toma entradas reales y booleanas y les asocia un conjunto de pesos, junto con un sesgo (el umbral que mencioné anteriormente). Aprendemos los pesos, obtenemos la función. Usemos un perceptrón para aprender una función OR. </p>

![Perceptron](https://drive.google.com/uc?export=view&id=1Kp8OYWpZbPX3PNkbsw154Q8KOqbdicuf)



![Perceptron Matematica](https://drive.google.com/uc?export=view&id=1_RhrJBSBSsjbpc89XGrX32VDec7WmLXA)


<p style='text-align: justify;'>
Como su nombre indica, el MLP es esencialmente una combinación de capas de perceptrones entrelazados. Utiliza las salidas de la primera capa como entradas de la siguiente capa hasta que finalmente, después de un número particular de capas, alcanza la capa de salida. Las capas entre las capas de entrada y salida se denominan capas ocultas. Al igual que con el perceptrón, MLP también tiene pesos que se deben ajustar para entrenar el sistema. Estos pesos ahora vienen en forma de matriz en cada unión entre capas. </p>



![MLP](https://drive.google.com/uc?export=view&id=15L2S9jFGi_j7E2KNK2p6Gno1_xKZbeQP)


<p style='text-align: justify;'>
La primera parte de la creación de un MLP es definir una topología y desarrollar el algoritmo feedforward. Feedforward es esencialmente el proceso utilizado para convertir la entrada en una salida. Sin embargo, no es tan simple como en el perceptrón, ya que ahora necesita iterar sobre varias capas. Usando operaciones matriciales</p>

Ahora definimos el MLP usando el framework de redes neuronales `Keras`. En este enfoque, una capa se puede declarar como la entrada de la siguiente capa al momento de definir la siguiente capa.

In [ ]:
from keras.layers import Dense, Input, Dropout
from keras.optimizers import SGD
from keras.models import Model
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [ ]:
# Defina la capa de entrada que tiene forma (, 7) y de tipo float32. Ninguno indica el número de instancias
input_layer = Input(shape=(7,), dtype='float32')

#### Funciones de activación

La función de activación no es más que una función matemática que toma una entrada y produce una salida. La función se activa cuando el resultado calculado alcanza el umbral especificado.
![Funcion Activacion](https://drive.google.com/uc?export=view&id=1w3slknVN9VGDmQzHDoPxhRCQZ1xFwmvN)



In [ ]:
#Las capas densas se definen con activación tanh.
dense1 = Dense(32, activation='tanh')(input_layer)
dense2 = Dense(16, activation='tanh')(dense1)
dense3 = Dense(16, activation='tanh')(dense2)

#### Dropout
<p style='text-align: justify;'>
Múltiples capas ocultas y una gran cantidad de neuronas en cada capa oculta les da a las redes neuronales la capacidad de modelar la no linealidad compleja de las relaciones subyacentes entre los regresores y el objetivo. Sin embargo, las redes neuronales profundas también pueden sobreajustar los datos de validación y dar malos resultados en la validación o el conjunto de pruebas. El dropout se ha utilizado efectivamente para regularizar redes neuronales profundas. En este ejemplo, se agrega una capa de Salida antes de la capa de salida. El dropout establece aleatoriamente p fracción de neuronas de entrada a cero antes de pasar a la siguiente capa. La eliminación aleatoria de entradas actúa esencialmente como un tipo de agrupamiento o metamodelo bagging.  Usamos p = 0.2 para abandonar el 20% de las características de entrada seleccionadas al azar.</p>

<p style='text-align: justify;'>
En el aprendizaje automático, la regularización es una forma de evitar el sobreajuste. La regularización reduce el sobreajuste al agregar una penalización a la función de pérdida. Al agregar esta penalización, el modelo se entrena de tal manera que no aprende pesos de conjunto de características interdependientes. Aquellos de ustedes que conocen la Regresión logística pueden estar familiarizados con las penalizaciones L1 (Laplaciana) y L2 (Gaussiana).</p>

![Dropout](https://drive.google.com/uc?export=view&id=1WwBQnaNmT2CfkPR439-uR_5pOy-kAluK)


El dropout obliga a una red neuronal a aprender características más robustas que son útiles junto con muchos subconjuntos aleatorios diferentes de las otras neuronas.

In [ ]:
dropout_layer = Dropout(0.2)(dense3)

In [ ]:
# Finalmente, la capa de salida da predicción para la presión de aire del día siguiente.
output_layer = Dense(1, activation='linear')(dropout_layer)

Las capas de entrada, densa y de salida ahora se empaquetarán dentro de un Modelo, que es una clase envolvente para entrenar y hacer predicciones. El box plot de pm2.5 muestra la presencia de valores atípicos. Por lo tanto, el error absoluto medio (MAE) se usa ya que las desviaciones absolutas sufren menos fluctuaciones en comparación con las desviaciones al cuadrado.

Los pesos de la red están optimizados por el algoritmo Adam. Adam representa la estimación del momento adaptativo y ha sido una opción popular para entrenar redes neuronales profundas. A diferencia del descenso de gradiente estocástico, Adam usa diferentes tasas de aprendizaje para cada peso y actualiza por separado lo mismo a medida que avanza el entrenamiento. La tasa de aprendizaje de un peso se actualiza con base en promedios móviles ponderados exponencialmente de los gradientes del peso y los gradientes al cuadrado.

### Loss Functions
<p style='text-align: justify;'>
Las máquinas aprenden mediante una función de pérdida. Es un método para evaluar qué tan bien el algoritmo específico modela los datos dados. Si las predicciones se desvían demasiado de los resultados reales, la función de pérdida arrojaría un número muy grande. Gradualmente, con la ayuda de alguna función de optimización, la función de pérdida aprende a reducir el error en la predicción. </p>

<p style='text-align: justify;'>
No existe una función de pérdida única para todos los algoritmos en el aprendizaje automático. Hay varios factores involucrados en la elección de una función de pérdida para un problema específico, como el tipo de algoritmo de aprendizaje automático elegido, la facilidad de calcular las derivadas y, en cierta medida, el porcentaje de valores atípicos en el conjunto de datos.
    </p>

<p style='text-align: justify;'>
    En términos generales, las funciones de pérdida se pueden clasificar en dos categorías principales según el tipo de tarea de aprendizaje con la que nos estamos ocupando: pérdidas de regresión y pérdidas de clasificación. En la clasificación, estamos tratando de predecir la salida del conjunto de valores categóricos finitos, es decir, dado un gran conjunto de datos de imágenes de dígitos escritos a mano, categorizándolos en uno de 0 a 9 dígitos. La regresión, por otro lado, trata de predecir un valor continuo, por ejemplo, dada la superficie del piso, el número de habitaciones, el tamaño de las habitaciones, predecir el precio de la habitación.</p>
    
**Regression Losses:**

1. Mean Square Error (L2) $MSE = \frac{\sum_{y=1}^n(y_i - \hat{y_i})^2}{n}$

Como su nombre indica, el error cuadrático medio se mide como el promedio de la diferencia cuadrática entre las predicciones y las observaciones reales. Solo le preocupa la magnitud promedio del error, independientemente de su dirección. Sin embargo, debido a la cuadratura, las predicciones que están muy lejos de los valores reales se penalizan fuertemente en comparación con las predicciones menos desviadas. Además, MSE tiene buenas propiedades matemáticas que facilitan el cálculo de gradientes.

2. Mean Absolute Error (L1) $MAE = \frac{\sum_{y=1}^n|y_i - \hat{y_i}|}{n}$

El error absoluto medio, por otro lado, se mide como el promedio de la suma de las diferencias absolutas entre las predicciones y las observaciones reales. Al igual que MSE, esto también mide la magnitud del error sin considerar su dirección. A diferencia de MSE, MAE necesita herramientas más complicadas como la programación lineal para calcular los gradientes. Además, MAE es más robusto para los valores atípicos, ya que no utiliza el cuadrado.

3. Mean Bias Error $MBE = \frac{\sum_{y=1}^n (y_i - \hat{y_i})}{n} $

**Classification Losses:**

1. Cross Entropy Loss = $ -(y_ilog(\hat(y_i)+(1-y_i)log(1-\hat{y_i})$

Tenga en cuenta que cuando la etiqueta real es 1 (y(i) = 1), la segunda mitad de la función desaparece, mientras que en caso de que la etiqueta real sea 0 (y (i) = 0), la primera mitad se descarta. En resumen, solo estamos multiplicando el registro de la probabilidad pronosticada real para la clase de verdad básica. Un aspecto importante de esto es que la pérdida de entropía cruzada penaliza fuertemente las predicciones que son confiables pero erróneas.

#### Metodos de optimización:

<p style='text-align: justify;'>
El aprendizaje profundo es un proceso iterativo. Con tantos parámetros para ajustar o métodos para probar, es importante poder entrenar modelos rápidamente, para completar rápidamente el ciclo iterativo. Esto es clave para aumentar la velocidad y la eficiencia de un equipo de aprendizaje automático.
De ahí la importancia de los algoritmos de optimización, como el descenso de gradiente estocástico, el descenso de gradiente de lote mínimo, el descenso de gradiente con impulso y el optimizador Adam.</p>
<p style='text-align: justify;'>
Estos métodos hacen posible que nuestra red neuronal aprenda. Sin embargo, algunos métodos funcionan mejor que otros en términos de velocidad. </p>


1. Mini-batch gradient descent: El descenso de gradiente tradicional necesita procesar todos los ejemplos de entrenamiento antes de realizar la primera actualización de los parámetros. En su lugar de eso, considere dividir el conjunto de prueba en conjuntos más pequeños. Cada conjunto pequeño se llama mini lote. Digamos que cada mini lote tiene 64 puntos de entrenamiento. ¡Entonces, podríamos entrenar el algoritmo en un mini lote a la vez y dar un paso una vez que se realice el entrenamiento para cada mini lote!

2. Gradient descent with momentum: El descenso de gradiente con momentum implica aplicar un suavizado exponencial al gradiente calculado. Esto acelerará el entrenamiento, porque el algoritmo oscilará menos hacia el mínimo y tomará más pasos hacia el mínimo. Por lo general, se utiliza un suavizado exponencial simple, lo que significa que hay dos hiperparámetros más para ajustar: la tasa de aprendizaje alfa y el parámetro de suavizado beta. Por lo general, este método casi siempre funciona mejor que el descenso de gradiente tradicional, y puede combinarse con el descenso de gradiente de mini lotes.

3. Adam significa: estimación adaptativa del momento. El método suaviza el gradiente, al igual que el momentum, pero utiliza un enfoque diferente. Se introducen 4 hyperparámetros. $\alpha, \beta?1 (0.9), \beta_2 (0.999), \epsilon$


$$S_{dw} = \beta_2S_{dw}+(1-\beta_2)dw^2 $$
<br>
$$S_{db} = \beta_2S_{db}+(1-\beta_2)db^2 $$

Entonces

$$ w:= w -\alpha\frac{dw}{\sqrt{S_{dw}+ \epsilon}}$$
<br>
$$ b:= b -\alpha\frac{db}{\sqrt{S_{db}+ \epsilon}}$$

Adam en Pseudocodigo
![Adam](https://drive.google.com/uc?export=view&id=1KtPjaukUkuztA3HsjnDj_5NbxglXsWFO)


#### Backpropagation

<p style='text-align: justify;'>
En el aprendizaje automático, la retropropagación es un algoritmo ampliamente utilizado en el entrenamiento de redes neuronales  para el aprendizaje supervisado. Las generalizaciones de la retropropagación existen para otras redes neuronales artificiales (ANN), y para funciones en general, una clase de algoritmos a los que se hace referencia genéricamente como "retropropagación". Al ajustar una red neuronal, la propagación hacia atrás calcula el gradiente de la función de pérdida con respecto a los pesos de la red para un solo ejemplo de entrada-salida, y lo hace de manera eficiente, a diferencia de un cálculo directo ingenuo del gradiente con respecto a cada peso individualmente. Esta eficiencia hace posible el uso de métodos de gradiente para entrenar redes multicapa, actualizar pesos para minimizar pérdidas; Descenso de gradiente, o variantes como el descenso de gradiente estocástico, se usan comúnmente. El algoritmo de retropropagación funciona calculando el gradiente de la función de pérdida con respecto a cada peso por la regla de la cadena, calculando el gradiente una capa a la vez, iterando hacia atrás desde la última capa para evitar cálculos redundantes de términos intermedios en la regla de la cadena; Este es un ejemplo de programación dinámica.</p>

![Backpropagation](https://drive.google.com/uc?export=view&id=1WZ7CuIk9B88wFtWAcEMsU8OleDVn9RgW)



In [ ]:
ts_model = Model(inputs=input_layer, outputs=output_layer)
ts_model.compile(loss='mean_absolute_error', optimizer='adam')
ts_model.summary()

El modelo se entrena llamando a la función de ajuste en el objeto modelo y pasando el X_train y el y_train. El entrenamiento se realiza para un número predefinido de epochs. Además, batch_size define el número de muestras del conjunto de entrenamiento que se utilizarán para una instancia de propagación inversa. El conjunto de datos de validación también se pasa para evaluar el modelo después de que se complete cada epochs. Un objeto ModelCheckpoint rastrea la función de pérdida en el conjunto de validación y guarda el modelo para el epochs, en la que la función de pérdida ha sido mínima.

In [ ]:
path = '/content/drive/My Drive/Cursos/Diplomado/Kafka/PrediccionPm2.5-MLP/models'
save_weights_at = os.path.join(path, 'PRSA_data_PM2.5_MLP_weights.{epoch:02d}-{val_loss:.4f}.hdf5')
save_best = ModelCheckpoint(save_weights_at, monitor='val_loss', verbose=0,
                            save_best_only=True, save_weights_only=False, mode='min',
                            period=1)
ts_model.fit(x=X_train, y=y_train, batch_size=16, epochs=30,
             verbose=1, callbacks=[save_best], validation_data=(X_val, y_val),
             shuffle=True)

## <font color='green'>**Actividad 2**</font>

Proponga, construya y entrene su propia red neuronal. (40 Minutos)

<font color='green'>**Fin Actividad 2**</font>

## <font color='green'>**Actividad 3**</font>

Evalue el resultado del modelo (30 minutos). Utilice
1. Mae
2. R2
3. Grafico de linea.

Se hacen predicciones para el pm2.5 del modelo mejor guardado. Las predicciones del modelo, que están en el pm2.5 escalado, se transforman inversamente para obtener predicciones del pm2.5 original.

<font color='green'>**Fin Actividad 3**</font>

<img src="https://drive.google.com/uc?export=view&id=1DNuGbS1i-9it4Nyr3ZMncQz9cRhs2eJr" width="100" align="left" title="Runa-perth">
<br clear="left">

## <font color='green'>**Metodos de series de tiempo con deep learning.**</font>

Las redes neuronales se utilizan en una amplia variedad de aplicaciones relacionadas con series temporales, debido a su capacidad para aprender patrones y relaciones a partir de datos secuenciales. Las series temporales son secuencias de datos medidos en intervalos de tiempo consecutivos, y se utilizan en muchos campos, incluyendo finanzas, medicina, ingeniería y ciencias sociales.

Aquí hay un resumen de las principales aplicaciones de las redes neuronales en series temporales, así como los conceptos clave utilizados:

Predicción de series temporales: Las redes neuronales se utilizan para predecir futuros puntos en una serie temporal basándose en datos históricos. Esto es útil en áreas como la previsión del mercado de valores, la planificación de la demanda y la predicción del tiempo.

Clasificación de series temporales: Las redes neuronales pueden aprender patrones en series temporales y clasificarlas en diferentes categorías. Por ejemplo, en el campo de la medicina, las series temporales de señales de electrocardiograma (ECG) pueden ser clasificadas como normales o anormales.

Anomalía en la detección de series temporales: Las redes neuronales se utilizan para detectar puntos anómalos en una serie temporal. Esto es útil en áreas como el monitoreo de maquinaria industrial para detectar fallos antes de que ocurran.

Conceptos clave en el uso de redes neuronales para series temporales:

Redes neuronales recurrentes (RNN): Son un tipo de red neuronal que tiene conexiones que se retroalimentan, lo que les permite mantener un estado interno y recordar información de pasos anteriores. Son especialmente adecuadas para el análisis de series temporales.

LSTM (Long Short-Term Memory): Es un tipo de RNN que utiliza celdas de memoria para mantener un estado a lo largo de largas secuencias, lo que les permite aprender patrones a largo plazo en series temporales.

GRU (Gated Recurrent Unit): Es otro tipo de RNN que utiliza compuertas para regular el flujo de información, similar a las LSTM pero con una estructura más simple.

Transformadores: Son una arquitectura de red neuronal que utiliza mecanismos de atención para capturar relaciones en secuencias de datos. Se utilizan para modelar series temporales con patrones complejos y a largo plazo.

Ventanas de tiempo: Al trabajar con series temporales, a menudo se dividen en ventanas de tiempo o subsecuencias para entrenar a la red neuronal en patrones más cortos.

Descomposición de series temporales: Se puede descomponer una serie temporal en componentes como tendencia, estacionalidad y ruido, lo que facilita el análisis y la predicción.

Embeddings temporales: A veces, se utilizan técnicas de incrustación para convertir series temporales en representaciones de baja dimensión que pueden ser más fácilmente analizadas por redes neuronales.

En resumen, las redes neuronales son herramientas poderosas para el análisis de series temporales, y se utilizan en una amplia gama de aplicaciones para predecir, clasificar y detectar anomalías en datos secuenciales.

<img src="https://drive.google.com/uc?export=view&id=1Igtn9UXg6NGeRWsqh4hefQUjV0hmzlBv" width="100" align="left" title="Runa-perth">
<br clear="left">

## <font color='red'>**Otras Arquitecturas de redes para series de datos.**</font>

### Red LSTM (Long Short-Term Memory):

Las LSTM son un tipo de redes neuronales recurrentes (RNN) diseñadas específicamente para evitar el problema de la desaparición del gradiente. Son esenciales para tareas de secuencia debido a su capacidad para recordar información a largo plazo. Aquí hay un breve resumen:

1. Memoria a Largo Plazo: A diferencia de las redes neuronales tradicionales, las LSTM tienen una estructura de memoria que les ayuda a recordar patrones o secuencias durante largos períodos, lo que las hace ideales para tareas relacionadas con series temporales y procesamiento del lenguaje natural.

2. Unidades de Puerta: Una característica clave de las LSTM son sus "puertas" (gate units). Estas puertas determinan qué información debe ser almacenada o descartada en la memoria celular. Hay tres tipos principales de puertas en una LSTM:

  a. Puerta de Entrada (Input Gate): Decide cuánta información nueva se almacenará en la memoria celular.

  b. Puerta de Olvido (Forget Gate): Decide cuánta información de la memoria celular actual se descartará.
  
  c. Puerta de Salida (Output Gate): Basándose en la memoria celular, decide cuál será el próximo estado oculto.

3. Flexibilidad en Secuencias: Las LSTM pueden manejar secuencias de entrada de diferentes longitudes sin necesidad de especificar la longitud de la secuencia con antelación.

4. Aplicaciones: Debido a su capacidad para recordar a largo plazo, las LSTM se utilizan en una variedad de aplicaciones, como traducción automática, generación de texto, análisis de sentimiento, series temporales, entre otros.

5. Variantes: Hay diversas variantes y extensiones de las LSTM, como las GRU (Gated Recurrent Units), que simplifican la arquitectura de las LSTM pero retienen la mayoría de sus beneficios.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generar datos
x = np.linspace(0, 50, 1000)
y = np.sin(x)

plt.plot(x, y)
plt.title("Serie de tiempo sinusoidal")
plt.show()


In [ ]:
def create_dataset(data, steps):
    X, Y = [], []
    for i in range(len(data)-steps-1):
        X.append(data[i:(i+steps)])
        Y.append(data[i + steps])
    return np.array(X), np.array(Y)

steps = 10
X, Y = create_dataset(y, steps)

# Reshape para [muestras, pasos de tiempo, características]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(30, activation='relu', input_shape=(steps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')


In [ ]:
model.fit(X, Y, epochs=50, verbose=1)


In [ ]:
y_pred = model.predict(X)

plt.figure(figsize=(15,6))
plt.plot(y, label='Verdadero')
plt.plot(np.arange(steps, 1000-1), y_pred, label='Predicho', alpha=0.7)
plt.title("Verdadero vs. Predicho")
plt.legend()
plt.show()
